In [1]:
import numpy as np

from random import randrange

from qiskit import BasicAer, IBMQ
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram

def initializeCostMatrixMemoization(totWeight, itemCount):
    global costMatrix;
    for ii in range(itemCount + 1):
        costMatrix.append([]);
        for jj in range(totWeight + 1):
            costMatrix[ii].append(-1);
    return;
    


In [2]:
def generateItems(*args):
    weightValueList    = [[],[]];
    itemCount   = args[0];
    minWeight   = args[1];
    maxWeight   = args[2];
    minValue    = args[3];
    maxValue    = args[4];
    valueQuant  = 1;
    weightQuant = 1;
    if(len(args) > 5):
        weightQuant     = args[5];
        if(len(args) == 7):
            valueQuant  = args[6];
    for ii in range(itemCount):
        tempWeight  = randrange(minWeight, maxWeight+1, weightQuant);
        weightValueList[0].append(tempWeight);
        tempValue   = randrange(minValue, maxValue+1, valueQuant);
        weightValueList[1].append(tempValue);
    return weightValueList;

def recursiveKnapsack(itemCount, remainingWeight):
    global costMatrix, backPackCap, itemsWeight, itemsValue;
    if ((itemCount == 0) or (remainingWeight <= 0)):
        return 0;
    if (costMatrix[itemCount][remainingWeight] >= 0):
        return costMatrix[itemCount][remainingWeight];
    if(itemsWeight[itemCount] > remainingWeight):
        costMatrix[itemCount][remainingWeight]  = recursiveKnapsack(itemCount - 1, remainingWeight);
    else:
        costMatrix[itemCount][remainingWeight]  = max(recursiveKnapsack(itemCount - 1, remainingWeight), recursiveKnapsack(itemCount - 1, remainingWeight - itemsWeight[itemCount]) + itemsValue[itemCount]);
    return costMatrix[itemCount][remainingWeight];

def backtraceItems(itemCount, totalCap):
    global costMatrix, itemsWeight, itemsValue;
    chosenItems = [];
    for ii in range(itemCount):
        chosenItems.append(0);
    itemIndex   = itemCount;
    weightIndex = totalCap;
    while(itemIndex > 0):
        if (costMatrix[itemIndex][weightIndex] == costMatrix[itemIndex-1][weightIndex]):
            itemIndex   -= 1;
        else:
            weightIndex -= itemsWeight[itemIndex];
            itemIndex   -= 1;
            chosenItems[itemIndex]  = 1;    
    return chosenItems;

#provider = IBMQ.load_account();

costMatrix  = [];
backPackCap = 2;
numOfItems  = 2;                # Number of objects

rangeMinWeight  = 1;
rangeMaxWeight  = 2;
rangeMinValue   = 10;
rangeMaxValue   = 50;

# For choosing A & B: 0 < B*max(itemValues) < A as described in Lucas article
A   = 250;                      # For quadratic modeling chosen at start
B   = 10;                       # For quadratic modeling chosen at start

initializeCostMatrixMemoization(backPackCap, numOfItems);

valuables   = generateItems(numOfItems, rangeMinWeight, rangeMaxWeight, rangeMinValue, rangeMaxValue);
itemsWeight = valuables[0];     # Weight of the objects
itemsWeight.insert(0, None);
itemsValue  = valuables[1];     # Value of the objects
itemsValue.insert(0, None);
recursiveKnapsack(numOfItems, backPackCap);
chosenItems = backtraceItems(numOfItems, backPackCap);

del itemsValue[0];
del itemsWeight[0];
print("Backpack Capacity = ", backPackCap);
print("Items Weight = ", itemsWeight);
print("Items Value = ", itemsValue);
"""print("Cost Matrix")
for ii in range(numOfItems+1):
    print(costMatrix[ii]);"""

print('Max Value = ', costMatrix[numOfItems][backPackCap]);
print("Chosen Items = ", chosenItems);

# Weight values matrix as described in Lucas article
W_matrix    = [];
# Weight values Transpose of  matrix
W_matrix_T  = [];
for ii in range(numOfItems):
    W_matrix_T.append(-itemsWeight[ii]);
for ii in range(backPackCap):
    W_matrix_T.append(ii+1);

W_matrix    =  np.array([W_matrix_T]);
W_matrix    = W_matrix.T;
AWWT        = A * W_matrix * W_matrix_T;

L_T   = [];                             # Transpose of the lambda vector
for ii in range(numOfItems):
    L_T.append(0);
for i in range(backPackCap):
    L_T.append(1);

L   = np.array([L_T]).T;                # Lambda vector
ALLT = A * L * L_T;

ALT = np.dot(2 * A, L_T);

# Items value vector
BV_vector   = [];

for ii in range(numOfItems):
    BV_vector.append(-itemsValue[ii] * B);
for ii in range(backPackCap):
    BV_vector.append(0);

# Variables vector: 'x' variables --> items, 'y' variables --> weights
z   = [];
for ii in range(numOfItems):
    z.append('x' + str(ii));
for ii in range(backPackCap):
    z.append('y' + str(ii));


linearCoeff     = ALT + BV_vector;
#linearCoeff     = linearCoeff.tolist();
quadraticCoeffs = ALLT + AWWT;

tempMax = 1000;
for ii in range(len(z)):
    if (tempMax < max(abs(quadraticCoeffs[ii]))):
        tempMax = max(abs(quadraticCoeffs[ii]));
        
penalty = tempMax**3;

quadraticCoeffDict  = {};
for ii in range(len(z)):
    for jj in range(len(z)):
        if ((ii >= numOfItems) and (jj >= numOfItems)):
            if (ii != jj):
                quadraticCoeffDict[(z[ii],z[jj])] = penalty;
            else:
                quadraticCoeffDict[(z[ii],z[jj])] = quadraticCoeffs[ii][jj];
        else:
            quadraticCoeffDict[(z[ii],z[jj])] = quadraticCoeffs[ii][jj];

#backend = BasicAer.get_backend('statevector_simulator')

qubo    = QuadraticProgram();
for ii in range(numOfItems + backPackCap):
    qubo.binary_var(z[ii]);

qubo.minimize(linear=linearCoeff, quadratic=quadraticCoeffDict);

print(qubo.export_as_lp_string());

"""qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'));

qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)"""

#quantumBackend  = QuantumInstance(provider.get_backend('ibmq_qasm_simulator'), shots = 2048, skip_qobj_validation=False);
backend = BasicAer.get_backend('statevector_simulator')
#qaoa_mes = QAOA(quantum_instance = quantumBackend);
#qaoa = MinimumEigenOptimizer(qaoa_mes);     # using QAOA
#qaoa_result = qaoa.solve(qubo);
#print(qaoa_result);

grover_optimizer = GroverOptimizer(4, num_iterations=5, quantum_instance=backend)
results = grover_optimizer.solve(qubo)
print("x={}".format(results.x))
print("fval={}".format(results.fval))

Backpack Capacity =  3
Items Weight =  [2, 2]
Items Value =  [34, 43]
Max Value =  43
Chosen Items =  [1, 1]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 340 x0 - 430 x1 + 500 y0 + 500 y1 + 500 y2 + [ 2000 x0^2 + 4000 x0*x1
      - 2000 x0*y0 - 4000 x0*y1 - 6000 x0*y2 + 2000 x1^2 - 2000 x1*y0
      - 4000 x1*y1 - 6000 x1*y2 + 1000 y0^2 - 6219476736 y0*y1
      - 6219476736 y0*y2 + 2500 y1^2 - 6219476736 y1*y2 + 5000 y2^2 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= y0 <= 1
 0 <= y1 <= 1
 0 <= y2 <= 1

Binaries
 x0 x1 y0 y1 y2
End



KeyboardInterrupt: 